In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Equi7Grid tile

A tile is a rectangular area defined by four Equi7Grid coordinates and is the smalles unit in the Equi7Grid system. A tile belongs to a specific Equi7Grid tiling scheme to subdivide the large projection extent.  

In the `equi7grid` package such a tile is realised via the `Equi7Tile` class. Its implementation extends the definition of `pytileproj`'s [`RasterTile`](https://tuw-geo.github.io/pytileproj/latest/guides/tile.html#raster-tile) class. A user does not need to know how to initialise an `Equi7Tile` object, since this is done internally and only `Equi7Tile` instances are provided via tiling system and grid methods. The next sections provide more details on how to use an `Equi7Tile` instance.

First, lets assume we performed a tile query on an `Equi7Grid` instance and we retrieved a single `Equi7Tile` object:

In [ ]:
from equi7grid import get_standard_equi7grid

e7grid = get_standard_equi7grid({"T6": 500})
e7tile = e7grid.EU.get_tile_from_name("EU_E048N012T6")



```python
e7tile = ...
```

The object can be printed to the console already revealing some attributes:

In [ ]:
e7tile

### Properties

Lets continue with inspecting the properties of this object. The first important property is the name of the tile instance, which is unique across all tiling systems of the Equi7Grid.

In [ ]:
e7tile.name

"EU" is the tiling system/projection/continent identifier, "E048" is the X coordinate of the lower-left tile corner (divided by 1e5), "N012" is the Y coordinate of the lower-left tile corner (divided by 1e5), and "T6" is the tiling level/identified. Some tiles in Europe and Asia might have negative coordinates, where the Y coordinate identifier then changes to "S".

Since the Equi7Grid was build for efficiently representing high-resolution raster imagery, it aims for reducing data redundancy as best as possible. Since geospatial raster imagery is usually shaped for a specific application over land or ocean, an `Equi7Grid` instance stores a land-sea mask to evaluate if a tile covers land or not. Thus, each Equi7 tile has a `covers_land` property, which can be used for filtering tiles. 

In [ ]:
e7tile.covers_land

The shape of the tile is defined by its width (in pixels) 

In [ ]:
e7tile.width

and its height (in pixels).

In [ ]:
e7tile.height

Direct shape access is possible with (height, width)

In [ ]:
e7tile.shape

The same can be done by using actual world system coordinates. The width is accessable through

In [ ]:
e7tile.x_size

and the height through

In [ ]:
e7tile.y_size

The sizes of each pixel can be easily retrieved via

In [ ]:
e7tile.x_pixel_size

and 

In [ ]:
e7tile.y_pixel_size

A very important thing of the relationship between pixel and world system coordinates is the anchor point or pixel origin, i.e. to what point in the pixel the coordinate refers to. By default the GDAL definition is used, which states that the origin is in the upper-left corner of the upper-left pixel. But with an `Equi7Tile`, you also have the option to choose between all other corner points and the pixel center. 

The aformentioned properties and the different possibilities for the pixel origin are illustrated in the graphic below.


```{image} /guides/imgs/raster_geometry.png
   :align: center
   :width: 60%
```


If you are interested in the corner points of the tile object, several properties help you to access the respective coordinates. For example, the lower-left corner, i.o.w. the first pixel in the last row, has the following coordinates:

In [ ]:
e7tile.ll_x, e7tile.ll_y

If you want to know the full coordinate extent (x_min, y_min, x_max, y_max), you can call

In [ ]:
e7tile.coord_extent

Please note that all these coordinates refer to the pixel origin, which has been chosen during class initialisation.
If you are interested in the full extent of the tile (bold black line in the image before), you can use

In [ ]:
e7tile.outer_boundary_extent

The corner points of the outer boundary are available via

In [ ]:
e7tile.outer_boundary_corners

The full range of 1-D coordinates in a certain direction along the edges of a tile can be retrieved with 

In [ ]:
e7tile.x_coords

and 

In [ ]:
e7tile.y_coords

These coordinates only refer to the first row or first column. If you are interested in a meshgrid of every coordinate contained within the tile you can use:

In [ ]:
e7tile.xy_coords

The boundary of the tile is also availabe on a higher-level, e.g. as a WKT string, OGR or shapely geometry.

In [ ]:
e7tile.boundary_wkt

If you have an array/raster and you want to add geospatial context you require [GDAL's affine geotransformation parameters](https://gdal.org/en/stable/tutorials/geotransforms_tut.html). These can be easily retrieved with

In [ ]:
e7tile.geotrans

Another important aspect in this regard is explicit projection information, which is accessible via

In [ ]:
e7tile.pyproj_crs

which returns a `pyproj.CRS` instance. These two objects offer the best interface with other geospatial libraries, e.g. `rasterio`.

Not all properties have been discussed here. Please take look at the documentation to explore the full range of offered functionality.

### Plotting

A very nice feature of a tile is that you can plot it on a map. Several keywords can help you to beautify your plot. First, we can simply try to plot the tile we have retrieved before.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))
e7tile.plot()

Since the full extent of the projection is chosen by default, we can try limit the extent with:

In [ ]:
import pyproj

plt.figure(figsize=(12, 12))
e7tile.plot(extent=(0, 35, 20, 55), extent_proj=pyproj.CRS.from_epsg(4326))

The plot function also allows to change the projection. We can try to define a new one, e.g. a Pseudo-Mercator projection, and use it for our new map.

In [ ]:
import pyproj

plt.figure(figsize=(12, 12))
e7tile.plot(
    label_tile=True,
    facecolor="green",
    edgecolor="blue",
    edgewidth=5,
    alpha=0.7,
    proj=3857,
    extent=(0, 35, 20, 55),
    extent_proj=pyproj.CRS.from_epsg(4326),
)

### Topological operations

When using the `Equi7Tile` object for subsequent processing, it might be helpful to check if geometries have a certain topological relation with the tile or not. In the next example we define several simple polygon geometries using `pytileproj`'s geometry objects (more details on these objects can be found [here]()):

In [ ]:
import shapely
from pytileproj import ProjGeom

inner_poly = shapely.Polygon(
    [(5200000, 1300000), (5400000, 1300000), (5400000, 1600000), (5200000, 1600000)]
)
inner_geom = ProjGeom(geom=inner_poly, crs=e7tile.pyproj_crs)

outer_poly = shapely.Polygon(
    [(6200000, 2300000), (6400000, 2300000), (6400000, 2600000), (6200000, 2600000)]
)
outer_geom = ProjGeom(geom=outer_poly, crs=e7tile.pyproj_crs)

touches_poly = shapely.Polygon(
    [(5400000, 1300000), (6400000, 1300000), (6400000, 1600000), (5400000, 1600000)]
)
touches_geom = ProjGeom(geom=touches_poly, crs=e7tile.pyproj_crs)

contains_poly = shapely.Polygon(
    [(4500000, 1100000), (6400000, 1100000), (6400000, 1900000), (4500000, 1900000)]
)
contains_geom = ProjGeom(geom=contains_poly, crs=e7tile.pyproj_crs)

overlaps_poly = shapely.Polygon(
    [(5200000, 1300000), (6400000, 1300000), (6400000, 1600000), (5200000, 1600000)]
)
overlaps_geom = ProjGeom(geom=overlaps_poly, crs=e7tile.pyproj_crs)

An Equi7 tile has four topological operators returning a boolean depending on if the relation is given or not: `intersects`, `overlaps`, `within`, and `touches`. Using the tile object and the polygons, we can evaluate all possible relations:

In [ ]:
(
    e7tile.intersects(inner_geom),
    e7tile.intersects(touches_geom),
    e7tile.intersects(outer_geom),
    e7tile.intersects(contains_geom),
    e7tile.intersects(overlaps_geom),
)

In [ ]:
(
    e7tile.overlaps(inner_geom),
    e7tile.overlaps(touches_geom),
    e7tile.overlaps(outer_geom),
    e7tile.overlaps(contains_geom),
    e7tile.overlaps(overlaps_geom),
)

In [ ]:
(
    e7tile.within(inner_geom),
    e7tile.within(touches_geom),
    e7tile.within(outer_geom),
    e7tile.within(contains_geom),
    e7tile.within(overlaps_geom),
)

In [ ]:
(
    e7tile.touches(inner_geom),
    e7tile.touches(touches_geom),
    e7tile.touches(outer_geom),
    e7tile.touches(contains_geom),
    e7tile.touches(overlaps_geom),
)

### Coordinate Conversions

An Equi7 tile also provides an interface for pixel to coordinate (`rc2xy`) and coordinate to pixel (`xy2rc`) conversions. 

In [ ]:
import numpy as np

rows, cols = np.meshgrid(np.arange(25, 30), np.arange(0, 25))
x_coords, y_coords = e7tile.rc2xy(rows.flatten(), cols.flatten())
x_coords, y_coords

In [ ]:
rows, cols = e7tile.xy2rc(x_coords, y_coords)
rows, cols

### Magic Methods

For some of the aforementioned functions a set of magic methods are available to enable a Pythonic usage of an Equi7 tile. This involves for instance the `within` check, which is internally called when using Pythons `in`.

In [ ]:
inner_geom in e7tile, outer_geom in e7tile

The `str` method returns the WKT representation of the boundary of the tile.

In [ ]:
str(e7tile)